<a href="https://colab.research.google.com/github/bsekiewicz/dw_matrix_road_sign/blob/master/notebooks/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import datetime
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
    )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [10]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 20ms/step - loss: 2.6104 - accuracy: 0.2851 - val_loss: 1.2736 - val_accuracy: 0.6136
Epoch 2/5
272/272 [==============================] - 5s 18ms/step - loss: 0.9343 - accuracy: 0.7019 - val_loss: 0.2402 - val_accuracy: 0.9244
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.4172 - accuracy: 0.8709 - val_loss: 0.1046 - val_accuracy: 0.9679
Epoch 4/5
272/272 [==============================] - 5s 18ms/step - loss: 0.2637 - accuracy: 0.9187 - val_loss: 0.0549 - val_accuracy: 0.9846
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1945 - accuracy: 0.9420 - val_loss: 0.0358 - val_accuracy: 0.9894


0.9757369614512472

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_1']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_2']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_3']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_1']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_2']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=params.get('epochs', 5),
      verbose=0,
    )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [30]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
    'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
    'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
    'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
    'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_1': 0.3238520222092685, 'dropout_cnn_block_2': 0.4615787754597186, 'dropout_cnn_block_3': 0.49259546343780997, 'dropout_dense_block_1': 0.4788210239528277, 'dropout_dense_block_2': 0.6572435460356422}
accuracy=0.9383220076560974
{'batch_size': 160.0, 'dropout_cnn_block_1': 0.34003972177592284, 'dropout_cnn_block_2': 0.46902541183361984, 'dropout_cnn_block_3': 0.3329380952234464, 'dropout_dense_block_1': 0.5412839503089455, 'dropout_dense_block_2': 0.5986143613121411}
accuracy=0.9358276724815369
{'batch_size': 160.0, 'dropout_cnn_block_1': 0.30336475836485316, 'dropout_cnn_block_2': 0.458256748276903, 'dropout_cnn_block_3': 0.41877392852994616, 'dropout_dense_block_1': 0.4049278700570215, 'dropout_dense_block_2': 0.47163052179922194}
accuracy=0.9433106780052185
{'batch_size': 140.0, 'dropout_cnn_block_1': 0.3546176409946763, 'dropout_cnn_block_2': 0.37997691789848154, 'dropout_cnn_block_3': 0.40956268531667894, 'dropout_dense_block_1': 0.35185286